In [5]:
import os
import sys

sys.path.append("/Users/gilmer/recommenders")

import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
import papermill as pm

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.common.notebook_utils import is_databricks
from reco_utils.dataset.criteo import load_spark_df
from reco_utils.dataset.spark_splitters import spark_random_split

# Setup MML Spark
if not is_databricks():
    # get the maven coordinates for MML Spark from databricks_install script
    from tools.databricks_install import MMLSPARK_INFO
    packages = [MMLSPARK_INFO["maven"]["coordinates"]]
    repo = MMLSPARK_INFO["maven"].get("repo")
    spark = start_or_get_spark(packages=packages, repository=repo)
    dbutils = None
    print("MMLSpark version: {}".format(MMLSPARK_INFO['maven']['coordinates']))

from mmlspark.train import ComputeModelStatistics
from mmlspark.lightgbm import LightGBMClassifier

MMLSpark version: com.microsoft.ml.spark:mmlspark_2.11:0.18.1


In [6]:
# Criteo data size, it can be "sample" or "full"
DATA_SIZE = "sample"

# LightGBM parameters
# More details on parameters: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
NUM_LEAVES = 32
NUM_ITERATIONS = 50
LEARNING_RATE = 0.1
FEATURE_FRACTION = 0.8
EARLY_STOPPING_ROUND = 10

# Model name
MODEL_NAME = 'lightgbm_criteo.mml'

In [8]:
!pwd

/Users/gilmer/Desktop/Springboard/recommendation_systems


In [9]:
raw_data = load_spark_df(size=DATA_SIZE, spark=spark, dbutils=dbutils)
# visualize data
raw_data.limit(2).toPandas().head()

 29%|██▉       | 2.50k/8.58k [01:16<03:05, 32.8KB/s]


EOFError: Compressed file ended before the end-of-stream marker was reached